## 1 Implementing the Quantum Fourier Transform in Qiskit

Group G

Lars Herbold
Moritz Rau

In [1]:
import numpy as np
from qiskit import QuantumCircuit

def reverse_range(n):
    return range(n -1, -1, -1)

def phase_shift(target, control):
    return np.pi / (2 ** (target - control))

#1.2 QFT acting on the first n qubits
def apply_qft(n, circuit):
    for j in reverse_range(n):
        circuit.h(j)
        for k in reverse_range(j):
            theta = phase_shift(j, k)
            circuit.cp(theta, k, j)
        circuit.barrier()

    for i in range(n // 2):
        circuit.swap(i, n - i - 1)
    
    circuit.barrier()

#1.3 Inverse QFT
def apply_iqft(n, circuit):
    for i in reverse_range(n // 2):
        circuit.swap(i, n - i - 1)
        
    circuit.barrier()
    
    for j in range(n):
        for k in range(j):
            theta = -phase_shift(j,k)
            circuit.cp(theta, k, j)
        circuit.h(j)
        circuit.barrier()

#1.4 Generalizing the functions 
def apply_qft_gen(qubits, circuit):
    n = len(qubits)
    
    for j in reverse_range(n):
        target_qubit = qubits[j]
        circuit.h(target_qubit)
        
        for k in reverse_range(j):
            control_qubit = qubits[k]
            theta = phase_shift(j,k)
            circuit.cp(theta, control_qubit, target_qubit)
            
    for i in range(n // 2):
        circuit.swap(qubits[i], qubits[n - i - 1])

def apply_iqft_gen(qubits, circuit):
    n = len(qubits)
    
    for i in reverse_range(n // 2):
        circuit.swap(qubits[i], qubits[n - i - 1])
        
    for j in range(n):
        target_qubit = qubits[j]
        for k in range(j):
            control_qubit = qubits[k]
            theta = -phase_shift(j,k)
            circuit.cp(theta, control_qubit, target_qubit)
        circuit.h(target_qubit)


# Print of QFT acting on the first n qbits
print("--- QFT n=3 ---")
n = 3
qc3 = QuantumCircuit(n)
apply_qft(n, qc3)
print(qc3)


print("--- QFT n=4 ---")
n = 4
qc4 = QuantumCircuit(n)
apply_qft(n, qc4)
print(qc4)

print("--- Inverse QFT n=4 ---")
n = 4
circuit = QuantumCircuit(n,0)
apply_iqft(n, circuit)
print(circuit)

--- QFT n=3 ---
                             ░                ░ ┌───┐ ░     ░ 
q_0: ───────────────■────────░───────■────────░─┤ H ├─░──X──░─
                    │        ░ ┌───┐ │P(π/2)  ░ └───┘ ░  │  ░ 
q_1: ──────■────────┼────────░─┤ H ├─■────────░───────░──┼──░─
     ┌───┐ │P(π/2)  │P(π/4)  ░ └───┘          ░       ░  │  ░ 
q_2: ┤ H ├─■────────■────────░────────────────░───────░──X──░─
     └───┘                   ░                ░       ░     ░ 
--- QFT n=4 ---
                                      ░                         ░      »
q_0: ────────────────────────■────────░────────────────■────────░──────»
                             │        ░                │        ░ ┌───┐»
q_1: ───────────────■────────┼────────░───────■────────┼────────░─┤ H ├»
                    │        │        ░ ┌───┐ │P(π/2)  │P(π/4)  ░ └───┘»
q_2: ──────■────────┼────────┼────────░─┤ H ├─■────────■────────░──────»
     ┌───┐ │P(π/2)  │P(π/4)  │P(π/8)  ░ └───┘                   ░      »
q_3: ┤ H ├─■────